In [1]:
# Prompt Engineering with GPT-2 and BERTScore - WORKING VERSION
# Run this in Google Colab

# First install all required packages
!pip install transformers torch bert-score pandas

# Import libraries
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from bert_score import score
import pandas as pd

# Part 1 - Load GPT-2 Model
print("Loading GPT-2 model...")
model_name = "gpt2"  # Using the smallest GPT-2 model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
print("Model loaded successfully!\n")

# Part 2 - Design Your Prompts and Generate Outputs
prompts = [
    # Direct instruction
    "Write a motivational quote about overcoming fear.",

    # Scenario-based
    "Imagine you're helping a friend who failed a test. Write something encouraging.",

    # Persona-based
    "As a wise monk, write a quote about inner strength.",

    # Keyword-based
    "Using the words 'growth', 'struggle', and 'hope', write something inspiring.",

    # Conversational
    "User: I feel like giving up. GPT-2: Here's a quote for you:"
]

def generate_text(prompt, num_outputs=3, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")

    outputs = []
    for _ in range(num_outputs):
        output = model.generate(
            inputs.input_ids,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7
        )
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        outputs.append(generated_text)

    return outputs

# Generate outputs for all prompts
print("Generating outputs for each prompt...")
all_outputs = {}
prompt_types = ["Direct", "Scenario", "Persona", "Keyword", "Conversational"]

for i, prompt in enumerate(prompts):
    all_outputs[prompt_types[i]] = generate_text(prompt)

# Display generated outputs
print("\n=== Generated Outputs ===")
for prompt_type, outputs in all_outputs.items():
    print(f"\nPrompt Type: {prompt_type}")
    for i, output in enumerate(outputs, 1):
        print(f"Output {i}: {output}")

# Part 3 - Human-Written Reference
human_reference = """
Success is not final, failure is not fatal: It is the courage to continue that counts.
- Winston Churchill
"""
print("\n=== Human Reference ===")
print(human_reference)

# Part 4 - Evaluate Outputs Using BERTScore
print("\nCalculating BERTScore (this may take a few minutes)...")

def calculate_bertscore(generated_texts, reference):
    # Make sure we're using CUDA if available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    P, R, F1 = score(generated_texts, [reference]*len(generated_texts),
                lang="en", verbose=True, device=device)
    return F1.tolist()

# Calculate BERTScore for all outputs
results = []
for prompt_type, outputs in all_outputs.items():
    f1_scores = calculate_bertscore(outputs, human_reference)
    for i, f1 in enumerate(f1_scores, 1):
        results.append({
            "Prompt Type": prompt_type,
            "Output #": i,
            "BERTScore F1": f"{f1:.4f}"
        })

# Part 5 - Results Table
print("\n=== BERTScore Results ===")
results_df = pd.DataFrame(results)
print(results_df)

# Save results
results_df.to_csv("gpt2_bertscore_results.csv", index=False)
print("\nResults saved to 'gpt2_bertscore_results.csv'")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 757.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.92 seconds, 1.57 sentences/sec

=== BERTScore Results ===
       Prompt Type  Output # BERTScore F1
0           Direct         1       0.8327
1           Direct         2       0.8381
2           Direct         3       0.8422
3         Scenario         1       0.8412
4         Scenario         2       0.8227
5         Scenario         3       0.8325
6          Persona         1       0.8405
7          Persona         2       0.8401
8          Persona         3       0.8262
9          Keyword         1       0.8319
10         Keyword         2       0.8306
11         Keyword         3       0.8397
12  Conversational         1       0.8249
13  Conversational         2       0.8235
14  Conversational         3       0.8270

Results saved to 'gpt2_bertscore_results.csv'
